![Image of Yaktocat](http://i.stack.imgur.com/aogj9.gif)
# Part1: Crawler, Store Server, Repository

In [1]:
# crawl webpages

import urllib2
import urlparse

def simplifyUrl(url):
    return urlparse.urlsplit(url).geturl()

def crawlWebpage(url):
    connection = urllib2.urlopen(url)
    page = connection.read()
    connection.close
    return page
testPageUrl = 'https://en.wikipedia.org/wiki/Lightweight_Java_Game_Library'
testPageId = 1234
testPage = crawlWebpage(testPageUrl)
print simplifyUrl(testPageUrl)

https://en.wikipedia.org/wiki/Lightweight_Java_Game_Library


In [2]:
# store pages in repository

import zlib
import urlparse
import struct

packetFormat = 'IxHI'
packetByteSize = 12

def checksum(url):
    import binascii
    return binascii.crc32(url) & 0xffffffff #this keeps the results consistent across all platforms

#packs a given webpage information into bytes
def packWebpage(webpage, url, docID):
    url = simplifyUrl(url)
    packet = struct.pack(packetFormat, docID, len(url), len(webpage)) + url + webpage
    return packet

#unpacks a given packet into webpage, url, and docID
def unpackWebpage(packet):
    info = struct.unpack(packetFormat, packet[:packetByteSize])#because 3 arguments
    docID, urlLength, docLength = info[:]
    url = packet[packetByteSize:packetByteSize + urlLength]
    webpage = packet[packetByteSize + urlLength:]
    return webpage, url, docID

#packs a webpage and saves it by docID. save name is subject to change in the future.
def storePage(webpage, url, docID):
    url = simplifyUrl(url)
    print("Struct Size: %i" %struct.calcsize(packetFormat))
    packet = packWebpage(webpage, url, docID)
    compressedPacket = zlib.compress(packet)
    output = open('poople\\page_repository\\' + urlparse.urlparse(url)[1] + '%i' % docID, 'w')
    output.write(compressedPacket)
    output.close()

packet = packWebpage(testPage, testPageUrl, testPageId)
storePage(testPage, testPageUrl, testPageId)

Struct Size: 12


# Indexer(Parsing part) and Lexicon

In [3]:
# parse and simplify page

from bs4 import BeautifulSoup
import time
import re
import string

def simplifyPage(page):
    before = time.clock()
    soup = BeautifulSoup(testPage, "lxml")
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    text = re.sub(r'\W+', ' ', text)
    after = time.clock()
    print("Simplify Page Time: %f" %(after-before))
    return text

def parseWordsInPage(simplifiedPage):
    before = time.clock()
    #words = re.split(r'\W+', simplifiedPage)
    words = simplifiedPage.split(' ')
    after = time.clock()
    print("Parse Time: %f" %(after-before))
    for word in words:
        if word == "":
            words.remove(word)
    return words


# test page simplification
simplifiedPage = simplifyPage(testPage)
wordList = parseWordsInPage(simplifiedPage)
for word in wordList:
    print word

Simplify Page Time: 0.127982
Parse Time: 0.000159
Lightweight
Java
Game
Library
Wikipedia
the
free
encyclopedia
Lightweight
Java
Game
Library
From
Wikipedia
the
free
encyclopedia
Jump
to
navigation
search
This
article
has
multiple
issues
Please
help
improve
it
or
discuss
these
issues
on
the
talk
page
Learn
how
and
when
to
remove
these
template
messages
The
topic
of
this
article
may
not
meet
Wikipedia
s
general
notability
guideline
Please
help
to
establish
notability
by
citing
reliable
secondary
sources
that
are
independent
of
the
topic
and
provide
significant
coverage
of
it
beyond
its
mere
trivial
mention
If
notability
cannot
be
established
the
article
is
likely
to
be
merged
redirected
or
deleted
Find
sources
Lightweight
Java
Game
Library
news
newspapers
books
scholar
JSTOR
free
images
March
2016
Learn
how
and
when
to
remove
this
template
message
This
article
relies
too
much
on
references
to
primary
sources
Please
improve
this
by
adding
secondary
or
tertiary
sources
August
2015
Learn
h

In [4]:
# TO DELETE

import sys
import time
import binascii

#The hash we are using. Can be changed without affecting the program
def wordHash(word):
    word = word.lower()
    return binascii.crc32(word) & 0xffffffff

#adds the word to the hashtable
#saves the word to the file so next time it will be loaded like normal
def addWordToLexicon(word, hashtable):
    word = word.lower()
    hashtable = addWordToHashtable(word, hashtable)
    saveNewWord(word)
    return hashtable

#adds word to given hashtable. Does not save to file. Hashes word
def addWordToHashtable(word, hashtable):
    word = word.lower()
    hashtable[wordHash(word)] = len(hashtable)
    return hashtable

#gets the wordID from the hashtable
def getWordId(word, hashtable):
    try:
        lowerWord = word.lower()
        return hashtable[wordHash(lowerWord)]
    except KeyError:
        print "Word not in lexicon: " + word
        return -1;

#saves the word to a file
def saveNewWord(word):
    word = word.lower()
    word = word + ","
    output = open('\\poople\\lexicon', 'a')
    output.write(word)
    output.close()
    
#loads the lexicon from file. creates the hashtable from lexicon
def loadLexicon():
    lexiconList = []
    stream = open('poople\\lexicon', 'r')
    lexicon = stream.read()
    stream.close()
    lexiconList = lexicon.split(",")
    hashtable = {}
    for word in lexiconList:
        if word != '':
            hashtable = addWordToHashtable(word, hashtable)
    return hashtable
    
wordHashtable = loadLexicon()
print len(wordHashtable)
addWordToHashtable("hello", wordHashtable)
print getWordId("hello", wordHashtable)
print len(wordHashtable)
print wordHashtable

1
1
2
{64952355L: 0, 907060870L: 1}


# Anchors, URL Resolver, Links, Doc Index, URL Server

# Indexer(rest) Barrels, Sorter, PageRank

In [5]:
#handle (en/de)coding hits

def encodePlainHit(cap, imp, pos):
    result = 0
    if(cap):
        result +=   0b1000000000000000 #2^16
    result += imp << 12
    result += pos & 0b0000111111111111
    return result
def encodeFancyHit(cap, ftype, pos):
    result = 0
    if(cap):
        result +=   0b1000000000000000 #2^16
    result += 0b0111000000000000 #fancy hits are always 7
    result += ftype << 8
    return result
def encodeAnchorHit(cap, ahash, pos):
    result = 0b0111000000000000
    if(cap):
        result +=   0b1000000000000000 #2^16
    result += pos
    result += ahash << 4
    return result

def decodeHit(hit):
    result = []
    result.append(hit >> 15)
    imp = (hit >> 12) & 0b111
    result.append(imp)
    if(imp != 7):
        result.append(hit & 0b0000111111111111)
    else: #fancy hit
        fancyType = (hit >> 8) & 0b1111
        result.append(fancyType)
        if(fancyType == 0): #anchor hit
            result.append(hit >> 4 & 0b1111)
            result.append(hit & 0b1111)
        else:
            result.append(hit & 0b11111111)
    return result

hit = encodePlainHit(False, 4, 0b001100000111)
fhit = encodeFancyHit(True, 0b0111, 0b10000000)
ahit = encodeAnchorHit(True, 0b1000, 0b1000)
print(decodeHit(hit))
print(decodeHit(fhit))
print(decodeHit(ahit))

[0, 4, 775]
[1, 7, 7, 0]
[1, 7, 0, 8, 8]


In [44]:
# start to put together hit lists

def parseTitleHits(page):
    hits = []
    soup = BeautifulSoup(page, "lxml")
    title = soup.title.string
    simplifiedTitle = re.sub(r'\W+', ' ', title).split(" ")
    result = ""
    for word in simplifiedTitle:
        capital = not word.islower()
        fontSize = 6
        result += encodeFancyHit(capital, 0, 0)
    return result

def encodeForBarrel(pageID, page):
    result = struct.pack("L", pageID)
    wordHits = {}
    wordNum = {}
    for word in page.split(" "):
        if(word in wordHits):
            wordHits[word] += struct.pack("L", encodePlainHit(1, 0, 0))
            wordNum[word] += 1
        else:
            wordHits[word] = struct.pack("L", wordHash(word))
            wordNum[word] = 1
    for key in wordHits:
        result += struct.pack("L", wordHash(key)) + struct.pack("H", wordNum[key]) + wordHits[key]
        print(key, wordHits[key], wordNum[key])
    return result

string = encodeForBarrel(101, simplifiedPage)

(u'', '\x00\x00\x00\x00\x00\x80\x00\x00', 2)
(u'code', '\x980\x15w\x00\x80\x00\x00', 2)
(u'expandedArticles', '@\x16\xac\x92', 1)
(u'Learn', '\\\x9c\x0eh\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00', 4)
(u'SAGE', '\xce\x19\xbf\x06', 1)
(u'existing', '\xc1^\xd48', 1)
(u'Elflight', '\x8a\x10\xaa\xc2', 1)
(u'NScripter', '\x87\xd8\xa7\xa6', 1)
(u'issues', '\x83\x7f}\xda\x00\x80\x00\x00', 2)
(u'poorly', '\xa6\xfa#u', 1)
(u'Foundation', '\x9a\x15\x93_', 1)
(u'iMUSE', '\xa6\xd8\xbc\xf0', 1)
(u'pagesPermanent', '\xb9\xdc\x12\xb2', 1)
(u'Torque', '\xfc|\x86\xc1', 1)
(u'to', '\xc4\xd2\x87\xd7\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00\x00\x80\x00\x00', 21)
(u'citing', '\xdd\xbcs ', 1)
(u'program', '\x84w\xed\x92', 1)
(u'GLFW', '\\\n\xb

# Ranking System, Searcher